# Описание Домашнего Задания

Технический анализ

<b>Цель:</b>

В данном домашнем задании вы потренируетесь в построении торговой стратегии на базе технического анализа. Также вы построите первую модель оценки эффективности торговой стратегии.


<b>Описание/Пошаговая инструкция выполнения домашнего задания:</b>

Уважаемый студент!

…Итак, данные собраны и можно начинать анализ. Коллеги посоветовали вам выделить паттерны и на их основе построить торговую стратегию.


Вы решили посмотреть сначала самые простые паттерны на основе скользящих средних и затем дополнить их более сложными шаблонами, такими как паттерны разворота, продолжения и свечного анализа.


Также вы понимаете, что нужен инструмент оценки эффективности и поскольку, у вас уже есть данные прошлых периодов, вы решаете создать инструмент оценки эффективности новой стратегии, пока у вас есть немного свободного времени.

На основании вышесказанного, вам необходимо построить модель, которая будет принимать решение о торговых операциях по одной или группе бумаг и криптовалют, и оценить ее эффективность на отложенных данных.


<b>Вам предлагается на основе представленной информации:</b>

1) Создать код на Python, который разделит ваши данные на тренировочный, тестовый и валидационный наборы данных.
2) Построить одну или несколько моделей на основе паттернов технического анализа, которая будет принимать торговые решения по бумагам SnP500 и/или криптовалютам.
3) Провести подбор гиперпараметров моделей с использованием обучающей и тестовой выборок.
4) Провести финальное тестирование построенных торговых стратегий на валидационном наборе данных и сравнить их между собой.
5) Сформировать дашборд, показывающий эффективность различных стратегий во времени.

# 1. Код на Python, разделяющий данные на тренировочный, тестовый и валидационный наборы данных

Принимаемые допущения:
- данные будем использовать из кэша (загрузка - отдельной, уже реализованной функцией);
- в данных должен быть ровно один тикер (поскольку рассматриваемые в курсе решения не добрались до управления портфелем, + вероятно это впоследствии возможно будет распараллелить);
- функционал разрабатываем на примере данных S&P500 ("^GSPC"), но делаем тикер управляемым параметром;
- границы train/test/valid датасетов должны быть управляемыми, для дальнейшего тестирования на различных временных периодах;
- в X_test / X_valid не должно быть "протекающих" признаков (Open / High / Low / Volume), при этом оставим возможность того, чтобы использовались другие признаки;
- вероятно, для сравнимости результатов тестирования длины тестовой и валидационной выборок должны быть одинаковыми, вне зависимости от длины обучающей (разные тикеры / разные периоды тестирования), поэтому реализация будет через определение временных границ, а не train_size / test_size из sklearn.model_selection_train_test_split;

In [1]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os

os.chdir(os.path.dirname(os.getcwd()))

import main
import logging

from typing import Optional

import pandas as pd

from src.core import utils

# initialize
logger = logging.getLogger()
# initialize config dict
config = main.main_launch()
# SUBSTITUTE FOR TESTING
# TODO

In [2]:
# Params for data download
TICKER = "^GSPC"
START_DT = '2010-01-01' # get as long as possible
END_DT = config.END_DT # defaults to today
INTERVAL = '1d'

# params for split function
train_start = None
train_end = "2023-01-01"
test_end = "2024-01-01"
valid_end = "2025-01-01" #config.END_DT

In [3]:
# Обновим данные в кэширующей базе
utils.update_tickers_data(
    tickers=TICKER, start_dt=START_DT, end_dt=END_DT, interval=INTERVAL
)

[INFO   ] 2025-03-22@17:05:24: Made sure table in database for interval='1d' exists
[INFO   ] 2025-03-22@17:05:24: Checking already available data...
[INFO   ] 2025-03-22@17:05:27: 0 tickers have no data at all
[INFO   ] 2025-03-22@17:05:27: 1 tickers lack history in start part
[INFO   ] 2025-03-22@17:05:27: 1 tickers lack history in end part
[INFO   ] 2025-03-22@17:05:27: Updating tickers lacking early history data...
[INFO   ] 2025-03-22@17:05:27: Downloading data using yfinance


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[ERROR  ] 2025-03-22@17:05:28: 
1 Failed download:
[ERROR  ] 2025-03-22@17:05:28: ['^GSPC']: YFPricesMissingError('possibly delisted; no price data found  (1d 2010-01-01 -> 2010-01-04 00:00:00)')
[INFO   ] 2025-03-22@17:05:28: Downloaded data shape: (0, 6)
[INFO   ] 2025-03-22@17:05:28: reshaped: (0, 7)
[INFO   ] 2025-03-22@17:05:28: Updating tickers lacking late history data...
[INFO   ] 2025-03-22@17:05:28: Downloading data using yfinance
[*********************100%***********************]  1 of 1 completed
[INFO   ] 2025-03-22@17:05:28: Downloaded data shape: (1, 5)
[INFO   ] 2025-03-22@17:05:28: reshaped: (1, 7)
[INFO   ] 2025-03-22@17:05:29: Data in caching DB updated


In [4]:
# Загрузим данные для дальнейшего использования
ticker_data = utils.get_history(
    tickers=TICKER,
    start=START_DT,
    end=END_DT,
    interval=INTERVAL,
    update_cache=False,
)

ticker_data.sample(3)

[INFO   ] 2025-03-22@17:05:31: Getting history from local cache DB...
[INFO   ] 2025-03-22@17:05:32: Got history of shape (3828, 7), 0 NaNs


,Date,Ticker,Open,Low,High,Close,Volume
545,2011-07-07 00:00:00.000000,^GSPC,1339.619995,1339.619995,1356.479980,1353.219971,4.069530e+09
2115,2017-10-02 00:00:00.000000,^GSPC,2521.199951,2520.399902,2529.229980,2529.120117,3.226370e+09
2745,2020-04-03 00:00:00.000000,^GSPC,2514.919922,2459.959961,2538.179932,2488.649902,6.096970e+09


In [5]:
def train_test_valid_split(
    ticker_data: pd.DataFrame,
    train_start: Optional[str],
    train_end: str,
    test_end: str,
    valid_end: str,
) -> tuple[
    pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame
]:
    """
    Split ticker data to training, testing and validation datasets
    """
    logger.info("Splitting ticker data to train/test/validation parts")
    # 0. Make sure that Date is ascending. Also reset index
    ticker_data["Date"] = pd.to_datetime(ticker_data["Date"])
    ticker_data = ticker_data.sort_values(by=["Date"], ascending=True).reset_index(
        drop=True
    )

    # 1. Drop leaky columns
    for col in ["Ticker", "Open", "Low", "High", "Volume"]:
        try:
            ticker_data.drop(columns=col, inplace=True)
        except:
            pass

    # 2. Perform train/test/valid split based on 'Date'
    # we don't need anything after validation end
    ticker_data = ticker_data[ticker_data["Date"] < valid_end].reset_index(drop=True)
    # in case train_start is defined - cut it
    if train_start is not None:
        ticker_data = ticker_data[ticker_data["Date"] >= train_start].reset_index(
            drop=True
        )
    # Train parts
    X_train = (
        ticker_data[ticker_data["Date"] < train_end]
        .drop(columns=["Close"])
        .reset_index(drop=True)
    )
    y_train = ticker_data[ticker_data["Date"] < train_end]["Close"].reset_index(
        drop=True
    )
    # Test parts
    X_test = (
        ticker_data[
            (ticker_data["Date"] >= train_end) & (ticker_data["Date"] < test_end)
        ]
        .drop(columns=["Close"])
        .reset_index(drop=True)
    )
    y_test = ticker_data[
        (ticker_data["Date"] >= train_end) & (ticker_data["Date"] < test_end)
    ]["Close"].reset_index(drop=True)
    # Validation parts
    X_val = (
        ticker_data[ticker_data["Date"] >= test_end]
        .drop(columns=["Close"])
        .reset_index(drop=True)
    )
    y_val = ticker_data[ticker_data["Date"] >= test_end]["Close"].reset_index(drop=True)

    return X_train, y_train, X_test, y_test, X_val, y_val

In [6]:
X_train, y_train, X_test, y_test, X_val, y_val = train_test_valid_split(
    ticker_data, train_start, train_end, test_end, valid_end
)

[INFO   ] 2025-03-22@17:05:38: Splitting ticker data to train/test/validation parts
